<a href="https://colab.research.google.com/github/muratyen/GoogleColab/blob/main/EuroCC_SparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Spark'ın sitesinden son versiyonunu indiriyoruz
!wget https://ftp.itu.edu.tr/Mirror/Apache/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

#İndirdiğimiz dosyayı extract ediyoruz
!tar -xvzf spark-3.1.1-bin-hadoop2.7.tgz

#Findspark paketini kuruyoruz
!pip install findspark

#SPARK_HOME parametresine spark folder path'i veriyoruz
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7/"

#Findspark'ı initialize ediyoruz
import findspark
findspark.init()

#Yeni bir spark session oluşturuyoruz
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.appName("PsSpark 3.0 on Google Colab").getOrCreate()

In [3]:
#Uplad ettiğimiz Iris.csv dosyasını okuyup bir dataframe objesi oluşturuyoruz
#Dosyamızın ilk satırı başlık olduğu için header parametresini true veriyoruz
#Veri tiplerini algılaması için inferSchema parametresini true veriyoruz
irisDF = sparkSession.read.csv(header = True, inferSchema = True, path = "/content/data/Iris.csv")

In [12]:
#Okuduğumuz dosyayı kontrol etmek için aşağıdaki metodu kullanıyoruz
#5 parametresi ilk 5 kaydın listesini ekrana yazdırmak için kullanılır. Bu parametre verilmezse default 20 kayıt gelir
irisDF.show(5)

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



In [15]:
import pyspark.ml

In [21]:
#Veri setimizdeki kolonları tek bir vektör kolonu haline getirmek için VectorAssembler kütüphanesini kullanacağız
#VectorAssembler kütüphanesini import ediyoruz
from pyspark.ml.feature import VectorAssembler

#Vektör kolonunda toplanacak tüm kolonların başlıklarını inputCols parametresi ile belirtiyoruz
#Yeni oluşacak vektör kolonunun başlık adını ise  outputCol parametresi ile belirtiyoruz
vectorAssembler = VectorAssembler(inputCols = ["Id", "SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol = "Features")

In [22]:
#Vektör kolonunu oluşturmak için irisDF i transform ediyoruz
irisDF = vectorAssembler.transform(irisDF)

In [25]:
irisDF.show(5)

+---+-------------+------------+-------------+------------+-----------+--------------------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|            Features|
+---+-------------+------------+-------------+------------+-----------+--------------------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|[1.0,5.1,3.5,1.4,...|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|[2.0,4.9,3.0,1.4,...|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|[3.0,4.7,3.2,1.3,...|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|[4.0,4.6,3.1,1.5,...|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|[5.0,5.0,3.6,1.4,...|
+---+-------------+------------+-------------+------------+-----------+--------------------+
only showing top 5 rows



In [28]:
#Veri setimizdeki string kolonunu sayısal bir label kolonu haline getirmek için StringIndexer kütüphanesini kullanacağız
#StringIndexer kütüphanesini import ediyoruz
from pyspark.ml.feature import StringIndexer

#String tipinde olan kolonunun başlığını inputCol parametresi ile belirtiyoruz
#Yeni oluşacak label kolonunun başlık adını ise outputCol parametresi ile belirtiyoruz
stringIndexer = StringIndexer(inputCol = "Species", outputCol = "Label")

In [29]:
#Label kolonunu oluşturmak için irisDF i transform ediyoruz
irisDF = stringIndexer.fit(irisDF).transform(irisDF)

In [33]:
#Diğer kolonlara ihtiyacımız olmadığı için sadece Features ve Label kolonlarını irisDF de bırakıyoruz 
irisDF = irisDF.select("Features", "Label")

In [37]:
#Decision tree algoritması çalıştırmak için DecisionTreeClassifier kütüphanesini kullanacağız
#DecisionTreeClassifier kütüphanesini import ediyoruz 
from pyspark.ml.classification import DecisionTreeClassifier

#Feature kolonunun başlığını setFeaturesCol metodu ile belirtiyoruz
#Label kolonunun başlığını setLabelCol metodu ile belirtiyoruz
decisionTreeClassifier = DecisionTreeClassifier().setFeaturesCol("Features").setLabelCol("Label")

In [38]:
#Decision tree modelimizi oluşturuyoruz
model = decisionTreeClassifier.fit(irisDF)